# <h1 style="font-size: 32px; font-weight: bold; color: black;">Final Report</h1>
## Analyzing the impact of environmental factors on public health (Year 2022)
### Summary 
This data science project aims to explore the relationship between hospital data of tuberculosis patients and the air quality in a regions of Bavaria and Brandenburg.Analyzing such data can provide valuable insights into potential correlations and
factors that might influence the spread of tuberculosis and compare between two regions.

### Main Questions
1. Do air quality metrics (such as PM2.5 and PM10) and the number of tuberculosis cases in the region have a statistically significant relationship?
2. Is there a vulnerable population that is particularly vulnerable to the effects of poor air quality on tuberculosis risk, such as age groups or those with pre-existing health disorders, and how can this knowledge guide public health interventions?
3. What are the potential peaks for tuberculosis incidence in connection to air quality, and how does the relationship between air quality and tuberculosis cases change throughout different geographic locations within the region?

### Datasources
#### Datasource1: Hospital Patients Dataset
* Metadata URL: https://www-genesis.destatis.de/genesis/online#astructure
* Data URL: https://www-genesis.destatis.de/genesis/online?operation=abruftabelleBearbeiten&levelindex=0&levelid=1699407973798&auswahloperation=abruftabelleAuspraegungAuswaehlen&auswahlverzeichnis=ordnungsstruktur&auswahlziel=werteabruf&code=23131-0011&auswahltext=&nummer=2&variable=2&name=GES055&werteabruf=Werteabruf#abreadcrumb
* Data Type: CSV

The dataset contains information about hospital patients, including their age, region, year of admission, gender, and whether they have confirmed tuberculosis (TB).

#### Datasource1: Air Quality Dataset
* Metadata URL: https://www.umweltbundesamt.de/en/data/air/air-data
* Data URL:  https://www.umweltbundesamt.de/api/air_data/v3/annualbalances/csv?component=1&year=2022&lang=en
* Data Type: CSV

The dataset contains information about air quality, including dust particle type, dust particle size and station location.

##  Install dependencies
Importing necessary libraries that will be used in the project

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import matplotlib.pyplot as plt
import datetime
from pathlib import Path

%matplotlib inline

##  Data Fetching and Preparation
Create a functionality to retrieve the data from the data source and channel it through the ETL (Extract, Transform, Load) pipeline for the purpose of constructing an Sqlite database.

In [17]:
def FetchData(file, deli):
    df = pd.read_csv(file, delimiter=deli)
    return df

def TransformAirQuality(file,column):
    extract_data = FetchData(file, ";")
    df = pd.DataFrame(extract_data)
    air_df = df[df['State / Measuring network'].isin(['Bavaria', 'Brandenburg'])].sort_values(by='Station code', ascending=True).reset_index(drop=True)
    if column:
        air_df = air_df.drop(columns='Number of daily mean values above 50 µg/m³')
        air_df = air_df.rename(columns={'Annual mean value in µg/m³': 'Annual P10 mean value in µg/m³'})
    return air_df

def TransformHospital(file, columns):
    extract_data = FetchData(file,",")
    df = pd.DataFrame(extract_data)
    df['TB-Diagnose'] = df['TB-Diagnose'].replace('-', 0)
    hos_df = df.rename(columns=columns)
    return hos_df

# Load data to create a SQLite file
def Load(df, table):
    current_directory = Path.cwd()
    parent_directory = current_directory.parent
    print(parent_directory)
    database_path = f'../data/{table}.sqlite'
    engine = create_engine(f"sqlite:///{database_path}")
    df.to_sql(table, engine, if_exists="replace")

def main():
    airquality_p2_file = "https://www.umweltbundesamt.de/api/air_data/v3/annualbalances/csv?component=9&year=2022&lang=en"
    airquality_p10_file = "https://www.umweltbundesamt.de/api/air_data/v3/annualbalances/csv?component=1&year=2022&lang=en"
    file_id = '1Nmqbq2YQOmVe0ncfwAHeOQDJRXQZVhlB'
    hospital_tb_file = f'https://drive.google.com/uc?id={file_id}'
    columns = {
    "Jahr" : "Year",
    "Bundesländer" : "State",
    "Geschlecht" : "Gender",
    "Insgesamt" : "Total_Patient_Admited"}
    data_p2 = TransformAirQuality(airquality_p2_file, False)
    data_p10 = TransformAirQuality(airquality_p10_file, True)
    data_hospital = TransformHospital(hospital_tb_file, columns)
    merged_df = pd.merge(data_p2, data_p10, on=["State / Measuring network","Station code", "Station name", "Station setting", "Station type"], how='inner')

    Load(merged_df, "Airquality")
    Load(data_hospital, "Hospital")

if __name__ == "__main__":
    main()

c:\Users\saad_\OneDrive\Desktop\Sem-1\MAD\made-template
c:\Users\saad_\OneDrive\Desktop\Sem-1\MAD\made-template


##  Load Data
Create a pandas dataframe using the local sqlite file.

In [19]:
def ConnDatabase(database_air):
    conn = sqlite3.connect(database_air)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_names = [table[0] for table in cursor.fetchall()]
    return conn, table_names

def SqlTableDataFrames(conn, table_names):
    dataframes = {}
    for table_name in table_names:
        query = f"SELECT * FROM {table_name};"
        df = pd.read_sql_query(query, conn)
        dataframes[table_name] = df
    return dataframes
    
# Close connection
def CloseConn(conn):
    conn.close()


# Connect DB for Airquality and hospital
database_air = r'..\data\Airquality.sqlite'
database_hos = r'..\data\Hospital.sqlite'
conn_air, air_table_names = ConnDatabase(database_air)
conn_hos, hos_table_names = ConnDatabase(database_air)

# Read the tables into Pandas DataFrames
dataframes_air = SqlTableDataFrames(conn_air, air_table_names)
dataframes_hos = SqlTableDataFrames(conn_hos, hos_table_names)

In [25]:
# airquality
for table_name, df in dataframes_air.items():
        print(f"Table: {table_name}")
        print(df.head())
        print('\n')

Table: Airquality
   index State / Measuring network Station code  \
0      0               Brandenburg      DEBB021   
1      1               Brandenburg      DEBB029   
2      2               Brandenburg      DEBB032   
3      3               Brandenburg      DEBB044   
4      4               Brandenburg      DEBB045   

                       Station name Station setting Station type  \
0                   Potsdam-Zentrum      urban area   background   
1                    Schwedt (Oder)   suburban area     industry   
2                  Eisenhüttenstadt   suburban area     industry   
3              Cottbus, Bahnhofstr.      urban area      traffic   
4  Frankfurt (Oder), Leipziger Str.      urban area      traffic   

   Annual mean value in µg/m³  Annual P10 mean value in µg/m³  
0                         9.9                            17.0  
1                        10.0                            15.0  
2                        12.0                            17.0  
3         

In [26]:
# hospital data
for table_name, df in dataframes_hos.items():
        print(f"Table: {table_name}")
        print(df.head())
        print('\n')

Table: Airquality
   index State / Measuring network Station code  \
0      0               Brandenburg      DEBB021   
1      1               Brandenburg      DEBB029   
2      2               Brandenburg      DEBB032   
3      3               Brandenburg      DEBB044   
4      4               Brandenburg      DEBB045   

                       Station name Station setting Station type  \
0                   Potsdam-Zentrum      urban area   background   
1                    Schwedt (Oder)   suburban area     industry   
2                  Eisenhüttenstadt   suburban area     industry   
3              Cottbus, Bahnhofstr.      urban area      traffic   
4  Frankfurt (Oder), Leipziger Str.      urban area      traffic   

   Annual mean value in µg/m³  Annual P10 mean value in µg/m³  
0                         9.9                            17.0  
1                        10.0                            15.0  
2                        12.0                            17.0  
3         

In [27]:
# Close DB Connection
CloseConn(conn_air)
CloseConn(conn_hos)